## Load Libraries and Data

In [1]:
import numpy as np
import pandas as pd
import modules.cleaning_utils as cut

In [2]:
complaint_data_csv = '../data/raw/NYPD_Complaint_Data_Historic_20241014.csv'

In [3]:
df = pd.read_csv(complaint_data_csv)
df

/tmp/ipykernel_26664/1619306799.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(complaint_data_csv)


,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,25640826H10611,12/05/2006,10:38:00,NaN,(null),NaN,12/05/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,(null),NaN,NaN,NaN,NaN,PATROL BORO QUEENS NORTH,(null),25-44,WHITE HISPANIC,M
1,16631501H8965,08/25/1973,12:15:00,NaN,(null),NaN,05/29/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,(null),NaN,NaN,NaN,NaN,PATROL BORO MAN NORTH,(null),45-64,BLACK,F
2,11118212H8714,04/22/2006,16:50:00,NaN,(null),NaN,04/22/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,(null),NaN,NaN,NaN,NaN,PATROL BORO BKLYN SOUTH,(null),45-64,BLACK,M
3,23471265H9399,07/20/2006,22:37:00,NaN,(null),NaN,07/20/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,(null),NaN,NaN,NaN,NaN,PATROL BORO BKLYN SOUTH,(null),18-24,BLACK,M
4,24565927H10142,04/14/2006,02:37:00,NaN,(null),NaN,04/14/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,(null),NaN,NaN,NaN,NaN,PATROL BORO BKLYN NORTH,(null),45-64,BLACK,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8914833,272431385,08/01/2023,03:45:00,08/01/2023,03:50:00,105.0,08/04/2023,341,PETIT LARCENY,321.0,...,U,NaN,40.697605,-73.727707,"(40.697605, -73.727707)",PATROL BORO QUEENS SOUTH,(null),65+,BLACK,M
8914834,273033152,08/16/2023,21:00:00,08/16/2023,21:05:00,105.0,08/17/2023,344,ASSAULT 3 & RELATED OFFENSES,101.0,...,F,NaN,40.718172,-73.736134,"(40.71817168981014, -73.73613374614645)",PATROL BORO QUEENS SOUTH,(null),25-44,BLACK,F
8914835,270041815,06/19/2023,01:50:00,06/19/2023,04:30:00,105.0,06/19/2023,107,BURGLARY,223.0,...,U,NaN,40.663804,-73.731601,"(40.663804, -73.731601)",PATROL BORO QUEENS SOUTH,(null),45-64,ASIAN / PACIFIC ISLANDER,M
8914836,273151416,08/19/2023,15:37:00,08/19/2023,15:52:00,113.0,08/19/2023,126,MISCELLANEOUS PENAL LAW,117.0,...,F,NaN,40.697672,-73.750627,"(40.697672, -73.750627)",PATROL BORO QUEENS SOUTH,(null),UNKNOWN,UNKNOWN,M


In [4]:
# Convert string ('null') values into nans
df = df.replace('(null)', np.nan)

# Capture original length and memory usage for later
original_length = len(df.index)
original_memory_usage = df.memory_usage().sum()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914838 entries, 0 to 8914837
Data columns (total 35 columns):
 #   Column             Dtype  
---  ------             -----  
 0   CMPLNT_NUM         object 
 1   CMPLNT_FR_DT       object 
 2   CMPLNT_FR_TM       object 
 3   CMPLNT_TO_DT       object 
 4   CMPLNT_TO_TM       object 
 5   ADDR_PCT_CD        float64
 6   RPT_DT             object 
 7   KY_CD              int64  
 8   OFNS_DESC          object 
 9   PD_CD              float64
 10  PD_DESC            object 
 11  CRM_ATPT_CPTD_CD   object 
 12  LAW_CAT_CD         object 
 13  BORO_NM            object 
 14  LOC_OF_OCCUR_DESC  object 
 15  PREM_TYP_DESC      object 
 16  JURIS_DESC         object 
 17  JURISDICTION_CODE  int64  
 18  PARKS_NM           object 
 19  HADEVELOPT         object 
 20  HOUSING_PSA        object 
 21  X_COORD_CD         float64
 22  Y_COORD_CD         float64
 23  SUSP_AGE_GROUP     object 
 24  SUSP_RACE          object 
 25  SUSP_SEX          

In [6]:
# Drop columns that are not interesting for our intended use
# Location descriptions are incomplete and not very useful
# PREM_TYP_DESC refers to the type of abode (apartment, store, etc.)
# PATROL_BORO is slightly different but mostly the same as BORO_NM which can join on other datasets better
# X/Y Coord and Lon_Lat are not needed with Latitude and Longitude columns already present
df.drop(['LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'PATROL_BORO',
         'Y_COORD_CD', 'X_COORD_CD', 'Lat_Lon'], axis = 1, inplace = True)

## Missing Values

In [7]:
df.isnull().sum()

CMPLNT_NUM                 0
CMPLNT_FR_DT             655
CMPLNT_FR_TM              48
CMPLNT_TO_DT         1818958
CMPLNT_TO_TM         1813518
ADDR_PCT_CD              771
RPT_DT                     0
KY_CD                      0
OFNS_DESC              18886
PD_CD                   7593
PD_DESC                 7593
CRM_ATPT_CPTD_CD         168
LAW_CAT_CD                 0
BORO_NM                 7884
JURIS_DESC                 0
JURISDICTION_CODE          0
PARKS_NM             8877278
HADEVELOPT           8883190
HOUSING_PSA          8244451
SUSP_AGE_GROUP       4583671
SUSP_RACE            3687178
SUSP_SEX             3820549
TRANSIT_DISTRICT     8716793
Latitude                 466
Longitude                466
STATION_NAME         8716793
VIC_AGE_GROUP        1623568
VIC_RACE                 671
VIC_SEX                  308
dtype: int64

In [8]:
# Complaint date and time occurrences are spread across 2 columns for exact vs. ending
# Can compress these values into one column to just capture date and time
df.CMPLNT_FR_DT = df.CMPLNT_FR_DT.fillna(df.CMPLNT_TO_DT)
df.CMPLNT_FR_TM = df.CMPLNT_FR_TM.fillna(df.CMPLNT_TO_TM)

# Drop remaining null values and superfluous date and time columns
df = df[~df['CMPLNT_FR_DT'].isnull()]
df = df[~df['CMPLNT_FR_TM'].isnull()]
df.drop(['CMPLNT_TO_DT', 'CMPLNT_TO_TM'], axis = 1, inplace = True)

In [9]:
# All of the missing location values appear to be in the same rows
df[df['Latitude'].isnull()]

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,...,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,25640826H10611,12/05/2006,10:38:00,NaN,12/05/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25-44,WHITE HISPANIC,M
1,16631501H8965,08/25/1973,12:15:00,NaN,05/29/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45-64,BLACK,F
2,11118212H8714,04/22/2006,16:50:00,NaN,04/22/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45-64,BLACK,M
3,23471265H9399,07/20/2006,22:37:00,NaN,07/20/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18-24,BLACK,M
4,24565927H10142,04/14/2006,02:37:00,NaN,04/14/2006,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45-64,BLACK,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8907414,268554247H1,05/19/2023,15:52:00,NaN,05/19/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,<18,BLACK,M,NaN,NaN,NaN,NaN,<18,BLACK,M
8909581,277525074H3,11/14/2023,07:05:00,NaN,11/14/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,45-64,BLACK,M,NaN,NaN,NaN,NaN,45-64,BLACK,M
8910318,269307857H1,06/04/2023,00:20:00,NaN,06/04/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,25-44,BLACK,M,NaN,NaN,NaN,NaN,25-44,BLACK,M
8912596,270206142H1,06/21/2023,16:25:00,NaN,06/21/2023,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,COMPLETED,...,25-44,BLACK,M,NaN,NaN,NaN,NaN,25-44,BLACK,M


In [10]:
# Remove null values
df = df[~df['Latitude'].isnull()]

In [11]:
df.isnull().sum()

CMPLNT_NUM                 0
CMPLNT_FR_DT               0
CMPLNT_FR_TM               0
ADDR_PCT_CD              305
RPT_DT                     0
KY_CD                      0
OFNS_DESC              18882
PD_CD                   7510
PD_DESC                 7510
CRM_ATPT_CPTD_CD         168
LAW_CAT_CD                 0
BORO_NM                 7883
JURIS_DESC                 0
JURISDICTION_CODE          0
PARKS_NM             8876654
HADEVELOPT           8882569
HOUSING_PSA          8243845
SUSP_AGE_GROUP       4583193
SUSP_RACE            3686801
SUSP_SEX             3820152
TRANSIT_DISTRICT     8716171
Latitude                   0
Longitude                  0
STATION_NAME         8716171
VIC_AGE_GROUP        1623480
VIC_RACE                 671
VIC_SEX                  308
dtype: int64

In [12]:
# Check for missing columns
# Highly missing: > 90% missingness
# Moderately missing: 1-90% missingness
# Low missing: < 1% missingness
highly_missing, moderately_missing, low_missing = cut.calculate_missing_ratios(df)

# 5 columns with high missingness
highly_missing

PARKS_NM            0.995787
HADEVELOPT          0.996450
HOUSING_PSA         0.924798
TRANSIT_DISTRICT    0.977783
STATION_NAME        0.977783
dtype: float64

In [13]:
# Drop these columns that are mostly null, as the data is irrecoverable
df.drop([*[x for x in df.columns if x in highly_missing.index.tolist()]], axis = 1, inplace = True)

In [14]:
# Some columns with moderate missingness
# All of these columns are age, sex and race columns
moderately_missing

SUSP_AGE_GROUP    0.514144
SUSP_RACE         0.413587
SUSP_SEX          0.428546
VIC_AGE_GROUP     0.182123
dtype: float64

In [15]:
# Validate age, sex, and race columns, converting invalid values to UNKNOWN or U
# Validate all columns while we are at it
df = cut.validate_age(df, ['SUSP_AGE_GROUP', 'VIC_AGE_GROUP'])
df = cut.validate_sex(df, ['SUSP_SEX', 'VIC_SEX'])
df = cut.validate_race(df, ['SUSP_RACE', 'VIC_RACE'])

In [16]:
# Rather than introduce biases via imputation, we will drop these records
low_missing

ADDR_PCT_CD         0.000034
OFNS_DESC           0.002118
PD_CD               0.000842
PD_DESC             0.000842
CRM_ATPT_CPTD_CD    0.000019
BORO_NM             0.000884
VIC_RACE            0.000075
VIC_SEX             0.000035
dtype: float64

In [17]:
# The rest of the missing values represent tiny amounts of the overall data (< 1% missingness)
# Drop missing values
low_missing_index = low_missing.index.tolist()

for c in low_missing_index:
    df = df[~df[c].isnull()]

In [18]:
df.isnull().sum()

CMPLNT_NUM           0
CMPLNT_FR_DT         0
CMPLNT_FR_TM         0
ADDR_PCT_CD          0
RPT_DT               0
KY_CD                0
OFNS_DESC            0
PD_CD                0
PD_DESC              0
CRM_ATPT_CPTD_CD     0
LAW_CAT_CD           0
BORO_NM              0
JURIS_DESC           0
JURISDICTION_CODE    0
SUSP_AGE_GROUP       0
SUSP_RACE            0
SUSP_SEX             0
Latitude             0
Longitude            0
VIC_AGE_GROUP        0
VIC_RACE             0
VIC_SEX              0
dtype: int64

In [19]:
# Data loss after all above cleaning
print(f'Data Loss: {(100 * (original_length - len(df.index)) / original_length):.3f} %')

Data Loss: 0.392 %


## Data Validation and Type Conversion

In [21]:
# Data runs from 2006 to 2023 according to the official documentation
# Validate dates and convert to datetime
df = cut.validate_dates_and_times(df, ['CMPLNT_FR_DT', 'RPT_DT'], ['CMPLNT_FR_TM'])

# According to maps, NY longitudes should be between -74.27 and -73.68, and latitudes between 40.49 and 40.92
df = cut.validate_coordinates(df)

/cdrive/Projects/Pace/Pace-Classwork/Scalable-Databases-CS673/Projects/Midterm/src/modules/cleaning_utils.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c] = pd.to_datetime(df[c])


In [22]:
# A number of offense codes have multiple descriptions
multi_ky_ofns = cut.multiple_descriptions(df, 'KY_CD', 'OFNS_DESC')
multi_ky_ofns

{116: ['FELONY SEX CRIMES', 'SEX CRIMES'],
 120: ['CHILD ABANDONMENT/NON SUPPORT', 'ENDAN WELFARE INCOMP'],
 124: ['KIDNAPPING',
  'KIDNAPPING & RELATED OFFENSES',
  'KIDNAPPING AND RELATED OFFENSES'],
 125: ['NYS LAWS-UNCLASSIFIED FELONY',
  'OTHER STATE LAWS (NON PENAL LA',
  'VEHICLE AND TRAFFIC LAWS'],
 343: ['OTHER OFFENSES RELATED TO THEF', 'THEFT OF SERVICES'],
 345: ['ENDAN WELFARE INCOMP', 'OFFENSES RELATED TO CHILDREN'],
 364: ['AGRICULTURE & MRKTS LAW-UNCLASSIFIED',
  'OTHER STATE LAWS (NON PENAL LA',
  'OTHER STATE LAWS (NON PENAL LAW)'],
 677: ['NYS LAWS-UNCLASSIFIED VIOLATION', 'OTHER STATE LAWS']}

In [24]:
# Replace the descriptions using self-chosen mapping
index_map = [0, 0, 2, 2, 1, 1, 0, 0]
cut.replace_description(df, multi_ky_ofns, index_map, 'KY_CD', 'OFNS_DESC')

In [25]:
# Checking the same for PD_CD and PD_DESC - only one description per code
multi_pd_desc = cut.multiple_descriptions(df, 'PD_CD', 'PD_DESC')
multi_pd_desc

{}

In [26]:
# Checking the same for JURISDICTION_CODE and JURIS_DESC - only one description per code
multi_pd_desc = cut.multiple_descriptions(df, 'JURISDICTION_CODE', 'JURIS_DESC')
multi_pd_desc

{}

In [27]:
# Define columns for type conversions
numeric_col = ['CMPLNT_NUM', 'ADDR_PCT_CD', 'KY_CD', 'PD_CD', 'JURISDICTION_CODE']
object_col = ['OFNS_DESC', 'PD_DESC', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'BORO_NM',
              'JURIS_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX',
              'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']

# Perform type conversions
df = cut.convert_to_categorical(df, numeric_col, object_col)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8859411 entries, 8 to 8914837
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         category      
 1   CMPLNT_FR_DT       datetime64[ns]
 2   CMPLNT_FR_TM       object        
 3   ADDR_PCT_CD        category      
 4   RPT_DT             datetime64[ns]
 5   KY_CD              category      
 6   OFNS_DESC          category      
 7   PD_CD              category      
 8   PD_DESC            category      
 9   CRM_ATPT_CPTD_CD   category      
 10  LAW_CAT_CD         category      
 11  BORO_NM            category      
 12  JURIS_DESC         category      
 13  JURISDICTION_CODE  category      
 14  SUSP_AGE_GROUP     category      
 15  SUSP_RACE          category      
 16  SUSP_SEX           category      
 17  Latitude           float64       
 18  Longitude          float64       
 19  VIC_AGE_GROUP      category      
 20  VIC_RACE           category  

In [29]:
# Compare loss of data and memory saved
print(f'Total Data Loss: {(100 * (original_length - len(df.index)) / original_length):.3f} %')
print(f'Total Memory Saved: {(original_memory_usage - df.memory_usage().sum()) / (1024 ** 3):.2f} GB')

Total Data Loss: 0.622 %
Total Memory Saved: 1.43 GB


In [30]:
# Write cleaned data to parquet file, split into two parts to abide by file limits for GitHub
df.iloc[:len(df.index) // 2].to_parquet('../data/cleaned/complaint_data_historic_cleaned_1.parquet.gz',
                                        index = False, compression = 'gzip')
# Write cleaned data to parquet file
df.iloc[len(df.index) // 2:].to_parquet('../data/cleaned/complaint_data_historic_cleaned_2.parquet.gz',
                                     index = False, compression = 'gzip')